In [ ]:
import cv2
import numpy as np
from skimage import color
import os
import numpy as np
import re
import torch
from torch import nn
from torchvision import transforms
from sklearn.linear_model import LinearRegression
from torchvision.io import read_image
from skimage.io import imread
import threading
import time
import pandas as pd
from datetime import datetime

# Evaluation Functions

In [ ]:
def calc_deltaE(source, target, color_chart_area):
  source = cv2.cvtColor(source, cv2.COLOR_BGR2RGB)
  target = cv2.cvtColor(target, cv2.COLOR_BGR2RGB)
  source = color.rgb2lab(source)
  target = color.rgb2lab(target)
  source = np.reshape(source, [-1, 3]).astype(np.float32)
  target = np.reshape(target, [-1, 3]).astype(np.float32)
  delta_e = np.sqrt(np.sum(np.power(source - target, 2), 1))
  return sum(delta_e) / (np.shape(delta_e)[0] - color_chart_area)

def calc_deltaE2000(source, target, color_chart_area):
  source = cv2.cvtColor(source, cv2.COLOR_BGR2RGB)
  target = cv2.cvtColor(target, cv2.COLOR_BGR2RGB)
  source = color.rgb2lab(source)
  target = color.rgb2lab(target)
  source = np.reshape(source, [-1, 3]).astype(np.float32)
  target = np.reshape(target, [-1, 3]).astype(np.float32)
  deltaE00 = deltaE2000(source, target)
  return sum(deltaE00) / (np.shape(deltaE00)[0] - color_chart_area)


def deltaE2000(Labstd, Labsample):
  kl = 1
  kc = 1
  kh = 1
  Lstd = np.transpose(Labstd[:, 0])
  astd = np.transpose(Labstd[:, 1])
  bstd = np.transpose(Labstd[:, 2])
  Cabstd = np.sqrt(np.power(astd, 2) + np.power(bstd, 2))
  Lsample = np.transpose(Labsample[:, 0])
  asample = np.transpose(Labsample[:, 1])
  bsample = np.transpose(Labsample[:, 2])
  Cabsample = np.sqrt(np.power(asample, 2) + np.power(bsample, 2))
  Cabarithmean = (Cabstd + Cabsample) / 2
  G = 0.5 * (1 - np.sqrt((np.power(Cabarithmean, 7)) / (np.power(
    Cabarithmean, 7) + np.power(25, 7))))
  apstd = (1 + G) * astd
  apsample = (1 + G) * asample
  Cpsample = np.sqrt(np.power(apsample, 2) + np.power(bsample, 2))
  Cpstd = np.sqrt(np.power(apstd, 2) + np.power(bstd, 2))
  Cpprod = (Cpsample * Cpstd)
  zcidx = np.argwhere(Cpprod == 0)
  hpstd = np.arctan2(bstd, apstd)
  hpstd[np.argwhere((np.abs(apstd) + np.abs(bstd)) == 0)] = 0
  hpsample = np.arctan2(bsample, apsample)
  hpsample = hpsample + 2 * np.pi * (hpsample < 0)
  hpsample[np.argwhere((np.abs(apsample) + np.abs(bsample)) == 0)] = 0
  dL = (Lsample - Lstd)
  dC = (Cpsample - Cpstd)
  dhp = (hpsample - hpstd)
  dhp = dhp - 2 * np.pi * (dhp > np.pi)
  dhp = dhp + 2 * np.pi * (dhp < (-np.pi))
  dhp[zcidx] = 0
  dH = 2 * np.sqrt(Cpprod) * np.sin(dhp / 2)
  Lp = (Lsample + Lstd) / 2
  Cp = (Cpstd + Cpsample) / 2
  hp = (hpstd + hpsample) / 2
  hp = hp - (np.abs(hpstd - hpsample) > np.pi) * np.pi
  hp = hp + (hp < 0) * 2 * np.pi
  hp[zcidx] = hpsample[zcidx] + hpstd[zcidx]
  Lpm502 = np.power((Lp - 50), 2)
  Sl = 1 + 0.015 * Lpm502 / np.sqrt(20 + Lpm502)
  Sc = 1 + 0.045 * Cp
  T = 1 - 0.17 * np.cos(hp - np.pi / 6) + 0.24 * np.cos(2 * hp) + \
      0.32 * np.cos(3 * hp + np.pi / 30) \
      - 0.20 * np.cos(4 * hp - 63 * np.pi / 180)
  Sh = 1 + 0.015 * Cp * T
  delthetarad = (30 * np.pi / 180) * np.exp(
    - np.power((180 / np.pi * hp - 275) / 25, 2))
  Rc = 2 * np.sqrt((np.power(Cp, 7)) / (np.power(Cp, 7) + np.power(25, 7)))
  RT = - np.sin(2 * delthetarad) * Rc
  klSl = kl * Sl
  kcSc = kc * Sc
  khSh = kh * Sh
  de00 = np.sqrt(np.power((dL / klSl), 2) + np.power((dC / kcSc), 2) +
                 np.power((dH / khSh), 2) + RT * (dC / kcSc) * (dH / khSh))
  return de00


def calc_mae(source, target, color_chart_area):
  source = np.reshape(source, [-1, 3]).astype(np.float32)
  target = np.reshape(target, [-1, 3]).astype(np.float32)
  source_norm = np.sqrt(np.sum(np.power(source, 2), 1))
  target_norm = np.sqrt(np.sum(np.power(target, 2), 1))
  norm = source_norm * target_norm
  L = np.shape(norm)[0]
  inds = norm != 0
  angles = np.sum(source[inds, :] * target[inds, :], 1) / norm[inds]
  angles[angles > 1] = 1
  f = np.arccos(angles)
  f[np.isnan(f)] = 0
  f = f * 180 / np.pi
  return sum(f) / (L - color_chart_area)



def calc_mse(source, target, color_chart_area):
  source = np.reshape(source, [-1, 1]).astype(np.float64)
  target = np.reshape(target, [-1, 1]).astype(np.float64)
  mse = sum(np.power((source - target), 2))
  return mse / ((np.shape(source)[
    0]) - color_chart_area)


def evaluate_cc(corrected, gt, color_chart_area, opt=1):
  """
    Color constancy (white-balance correction) evaluation of a given corrected
    image.
    :param corrected: corrected image
    :param gt: ground-truth image
    :param color_chart_area: If there is a color chart in the image, that is
     masked out from both images, this variable represents the number of pixels
     of the color chart.
    :param opt: determines the required error metric(s) to be reported.
         Options:
           opt = 1 delta E 2000 (default).
           opt = 2 delta E 2000 and mean squared error (MSE)
           opt = 3 delta E 2000, MSE, and mean angular eror (MAE)
           opt = 4 delta E 2000, MSE, MAE, and delta E 76
    :return: error(s) between corrected and gt images
    """

  if opt == 1:
    return calc_deltaE2000(corrected, gt, color_chart_area)
  elif opt == 2:
    return calc_deltaE2000(corrected, gt, color_chart_area), calc_mse(
      corrected, gt, color_chart_area)
  elif opt == 3:
    return calc_deltaE2000(corrected, gt, color_chart_area), calc_mse(
      corrected, gt, color_chart_area), calc_mae(corrected, gt,
                                                 color_chart_area)
  elif opt == 4:
    return calc_deltaE2000(corrected, gt, color_chart_area), calc_mse(
      corrected, gt, color_chart_area), calc_mae(
      corrected, gt, color_chart_area), calc_deltaE(corrected, gt,
                                                    color_chart_area)
  else:
    raise Exception('Error in evaluate_cc function')




def get_metadata(fileName, set, metadata_baseDir=''):
    """
    Gets metadata (e.g., ground-truth file name, chart coordinates and area).
    :param fileName: input filename
    :param set: which dataset?--options includes: 'RenderedWB_Set1',
      'RenderedWB_Set2', 'Rendered_Cube+'
    :param metadata_baseDir: metadata directory (required for Set1 only)
    :return: metadata for a given image
    evaluation_examples.py provides some examples of how to use it
    """

    fname, file_extension = os.path.splitext(fileName)  # get file parts
    name = os.path.basename(fname)  # get only filename without the directory

    if set == 'RenderedWB_Set1': # Rendered WB dataset (Set1)
        metadatafile_color = name + '_color.txt' # chart's colors info.
        metadatafile_mask = name + '_mask.txt' # chart's coordinate info.
        # get color info.
        f = open(os.path.join(metadata_baseDir, metadatafile_color), 'r')
        C = f.read()
        colors = np.zeros((3, 24))  # color chart colors
        temp = re.split(',|\n', C)
        # 3 x 24 colors in the color chart
        colors = np.reshape(np.asfarray(temp[:-1], float), (24, 3)).transpose()
        # get coordinate info
        f = open(os.path.join(metadata_baseDir, metadatafile_mask), 'r')
        C = f.read()
        temp = re.split(',|\n', C)
        # take only the first 4 elements (i.e., the color chart coordinates)
        temp = temp[0:4]
        mask = np.asfarray(temp, float)  # color chart mask coordinates
        # get ground-truth file name
        seperator = '_'
        temp = name.split(seperator)
        gt_file = seperator.join(temp[:-2])
        gt_file = gt_file + '_G_AS.png'
        # compute mask area
        mask_area = mask[2] * mask[3]
        # final metadata
        data = {"gt_filename": gt_file, "cc_colors": colors, "cc_mask": mask,
                "cc_mask_area": mask_area}

    elif set == 'RenderedWB_Set2': # Rendered WB dataset (Set2)
        data = {"gt_filename": name + file_extension, "cc_colors": None,
                "cc_mask": None, "cc_mask_area": 0}

    elif set == 'Rendered_Cube+': # Rendered Cube+
        # get ground-truth filename
        temp = name.split('_')
        gt_file = temp[0] + file_extension
        mask_area = 58373  # calibration obj's area is fixed over all images
        data = {"gt_filename": gt_file, "cc_colors": None, "cc_mask": None,
                "cc_mask_area": mask_area}
    else:
        raise Exception(
            "Invalid value for set variable. " +
            "Please use: 'RenderedWB_Set1', 'RenderedWB_Set2', 'Rendered_Cube+'")

    return data


# Prepare data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !unzip -q /content/drive/MyDrive/CC/datasets/Set1_ground_truth_images_wo_CC.zip -d Set1_ground_truth_images_wo_CC
# !unzip -q /content/drive/MyDrive/CC/datasets/Set1_input_images_wo_CC_JPG.zip -d Set1_input_images_wo_CC_JPG
# !unzip -q /content/drive/MyDrive/CC/datasets/Set1_input_images_metadata.zip -d Set1_input_images_metadata

In [ ]:
CHECKPOINTS = '/content/drive/MyDrive/CC/checkpoints'
IN_DIR = '/content/Set1_input_images_wo_CC_JPG'
GT_DIR = '/content/Set1_ground_truth_images_wo_CC'
FOLD_DIR = '/content/drive/MyDrive/CC/datasets/Set1_folds'
METADATA_DIR = '/content/Set1_input_images_metadata'

In [ ]:
!ls $CHECKPOINTS

a-unet_aug_20220918_104239_150.pth  sa_unet_aug_20220918_021432_5.pth
sa-unet_20220917_155653_75.pth	    scmn_20220925_111650_150.pth
sa_unet_aug_20220918_021432_35.pth


# Deep learning model

## Simple Color Mapping Network 

In [ ]:
class ConvBlock(nn.Module):

    def __init__(self, in_channels, out_channels):
        super(ConvBlock, self).__init__()

        # number of input channels is a number of filters in the previous layer
        # number of output channels is a number of filters in the current layer
        # "same" convolutions
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=True),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=True),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        x = self.conv(x)
        return x

class Kernel(nn.Module):
  def forward(self, x):
    r = x[:,0,:,:]
    g = x[:,1,:,:]
    b = x[:,2,:,:]
    return torch.stack([r,g,b,r**2,g**2,b**2,r*g,g*b,r*b,r*g*b,torch.ones_like(r)], dim=1)

class SCMN(nn.Module):
  def __init__(self, in_channels=3):
    super().__init__()
    self.Conv1 = ConvBlock(in_channels, 24)
    self.Conv2 = ConvBlock(24, 48)
    self.Conv3 = ConvBlock(48, 96)
    self.Conv4 = ConvBlock(96, 192)
    self.Conv5 = ConvBlock(192, 384)
    self.MaxPool = nn.MaxPool2d(2, 2)
    self.Flatten = nn.Flatten()
    self.Linear = nn.Linear(24576, 33)
    self.Kernel = Kernel()

  # Infer mode
  def forward(self, x):
    x2 = self.MaxPool(self.Conv1(x))
    x2 = self.MaxPool(self.Conv2(x2))
    x2 = self.MaxPool(self.Conv3(x2))
    x2 = self.MaxPool(self.Conv4(x2))
    x2 = self.MaxPool(self.Conv5(x2))
    x2 = self.Flatten(x2)
    x2 = self.Linear(x2)
    x2 = torch.reshape(x2, (-1, 3, 11)) # B x 3 x 11

    # x3 = self.Kernel(x) # B x 11 x 256 x 256
    # x3 = torch.reshape(x3, (-1, 11, 256 * 256)) # B x 11 x 65536
    
    # x4 = torch.bmm(x2, x3) # B x 3 x 65536
    # x5 = torch.reshape(x4, (-1, 3, 256, 256))

    return x2

# Inference model

## DL model

In [ ]:
mytransform = transforms.Compose([
    transforms.Resize((256, 256))
])

device = 'cuda'

model_checkpoint_path = os.path.join(CHECKPOINTS, 'scmn_20220925_111650_150.pth')
net = SCMN()
net.load_state_dict(torch.load(model_checkpoint_path))
net.eval()
net = net.to(device)

def outOfGamutClipping(I):
    """ Clips out-of-gamut pixels. """
    I[I > 1] = 1  # any pixel is higher than 1, clip it to 1
    I[I < 0] = 0  # any pixel is below 0, clip it to 0
    return I

def get_mapping_matrix(img):
  img = torch.tensor(img, dtype=torch.float32)
  img = torch.permute(img, (2, 0, 1))
  img = torch.unsqueeze(img, dim=0)
  img = img.to('cuda')
  with torch.no_grad():
    img = mytransform(img)
    out = net(img) # 1 x 3 x 11
  out = torch.squeeze(out) # 3 x 11
  mapping_matrix = out.cpu().numpy()
  return mapping_matrix

def kernel(img):
  r = img[:,:,0]
  g = img[:,:,1]
  b = img[:,:,2]
  return np.stack([r,g,b,r**2,g**2,b**2,r*g,g*b,r*b,r*g*b, np.ones_like(r)], axis=0) # 11 x height x width

def infer(img):
  M = get_mapping_matrix(img) # 3 x 11
  height, width, _ = img.shape
  img = kernel(img) # 11 x height x width
  img = img.reshape(11, -1) # (11 x width*height)
  img = np.matmul(M, img) # 3 x width * height
  img = img.reshape(3, height, width) # 3 x height x width
  img = np.transpose(img, (1,2,0)) # height x width x 3
  img = outOfGamutClipping(img)
  return img

# Eval

In [ ]:
def eval(name, imgin, gtimg):
    # read the image
    I_in = cv2.imread(imgin, cv2.IMREAD_COLOR)
    # read gt image
    gt = cv2.imread(gtimg, cv2.IMREAD_COLOR)
    # metadata
    metadata = get_metadata(name, 'RenderedWB_Set1', METADATA_DIR)  

    # white balance I_in
    I_corr = (infer(I_in[:,:,::-1] / 255)[:,:,::-1]*255).astype("uint8")  

    # import matplotlib.pyplot as plt
    # plt.figure(figsize=(20,10))
    # plt.subplot(1,3,1)
    # plt.imshow(I_in[:,:,::-1])
    # plt.title("Input")
    # plt.subplot(1,3,2)
    # plt.imshow(gt[:,:,::-1])
    # plt.title("Target")
    # plt.subplot(1,3,3)
    # plt.imshow(I_corr[:,:,::-1])
    # plt.title("Corrected")
    # plt.show()

    # Evaluation
    deltaE00, MSE, MAE, deltaE76 = evaluate_cc(I_corr, gt, metadata["cc_mask_area"],
                                            opt=4)

    return deltaE00, MSE.item(), MAE, deltaE76

In [ ]:
class EvalThread(threading.Thread):
  eval_log = pd.DataFrame(columns=["img_in", "img_gt", "deltaE2000", "MSE", "MAE", "deltaE76"])

  def __init__(self, part, n_part, imgs):
    super().__init__()
    self.part = part
    self.n_part = n_part
    self.imgs = imgs

  def run(self):
    imgs = self.imgs
    df = EvalThread.eval_log
    for i in range(len(imgs)):
      if i % self.n_part == self.part:
        img_in = os.path.join(IN_DIR, imgs[i] + '.jpg')
        img_gt = os.path.join(GT_DIR, imgs[i].rsplit('_', maxsplit=2)[0] + '_G_AS.png')
        deltaE00, MSE, MAE, deltaE76 = eval(imgs[i], img_in, img_gt)
        df.loc[len(df)] = {
            "img_in": img_in, 
            "img_gt": img_gt,
            "deltaE2000": deltaE00,
            "MSE": MSE,
            "MAE": MAE,
            "deltaE76": deltaE76
        }
        print(f'[AVG {len(df)}/{len(imgs)}] DeltaE 2000: {df["deltaE2000"].mean():.2f}, MSE={df["MSE"].mean():.2f}, MAE={df["MAE"].mean():.2f}, DeltaE 76={df["deltaE76"].mean():.2f}')


In [ ]:
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

imgs = []    
with open(os.path.join(FOLD_DIR, 'fold_1.txt'), 'r') as f:
    imgs += [s.strip().split(".")[0] for s in f.readlines()]

n_part = 8
threads = []
for part in range(n_part):
  thread = EvalThread(part, n_part, imgs)
  threads.append(thread)

for thread in threads:
  thread.start()
  thread.join()


[AVG 1/21046] DeltaE 2000: 5.84, MSE=137.40, MAE=8.52, DeltaE 76=6.40
[AVG 2/21046] DeltaE 2000: 5.15, MSE=123.14, MAE=5.09, DeltaE 76=5.72
[AVG 3/21046] DeltaE 2000: 4.87, MSE=109.81, MAE=4.08, DeltaE 76=5.49
[AVG 4/21046] DeltaE 2000: 4.38, MSE=86.66, MAE=4.38, DeltaE 76=4.81
[AVG 5/21046] DeltaE 2000: 6.38, MSE=137.79, MAE=6.19, DeltaE 76=7.47
[AVG 6/21046] DeltaE 2000: 5.91, MSE=122.72, MAE=5.79, DeltaE 76=6.99
[AVG 7/21046] DeltaE 2000: 5.57, MSE=113.03, MAE=5.64, DeltaE 76=6.77
[AVG 8/21046] DeltaE 2000: 5.23, MSE=100.43, MAE=5.35, DeltaE 76=6.27
[AVG 9/21046] DeltaE 2000: 5.13, MSE=98.95, MAE=5.18, DeltaE 76=6.20
[AVG 10/21046] DeltaE 2000: 4.87, MSE=91.66, MAE=4.88, DeltaE 76=5.86
[AVG 11/21046] DeltaE 2000: 4.90, MSE=95.79, MAE=4.72, DeltaE 76=5.95
[AVG 12/21046] DeltaE 2000: 4.91, MSE=96.88, MAE=4.69, DeltaE 76=5.98
[AVG 13/21046] DeltaE 2000: 4.97, MSE=105.49, MAE=4.55, DeltaE 76=6.05
[AVG 14/21046] DeltaE 2000: 4.88, MSE=99.80, MAE=4.55, DeltaE 76=5.92
[AVG 15/21046] DeltaE

In [ ]:
EVALUATION_SAVE_PATH = '/content/drive/MyDrive/CC/evals/'
FILE_NAME = f'evaluation_results_{timestamp}.csv'

fp = os.path.join(EVALUATION_SAVE_PATH, FILE_NAME)
EvalThread.eval_log.to_csv()
print("Saved evaluation results to ", fp)

Saved evaluation results to  /content/drive/MyDrive/CC/evals/evaluation_results_20220927_013820.csv


In [ ]:
EvalThread.eval_log

,img_in,img_gt,deltaE2000,MSE,MAE,deltaE76
0,/content/Set1_input_images_wo_CC_JPG/NikonD40_...,/content/Set1_ground_truth_images_wo_CC/NikonD...,5.836537,137.404310,8.516401,6.401685
1,/content/Set1_input_images_wo_CC_JPG/Canon600D...,/content/Set1_ground_truth_images_wo_CC/Canon6...,4.462629,108.881064,1.665026,5.035117
2,/content/Set1_input_images_wo_CC_JPG/Canon600D...,/content/Set1_ground_truth_images_wo_CC/Canon6...,3.813889,44.347317,4.847858,4.846437
3,/content/Set1_input_images_wo_CC_JPG/Canon1DsM...,/content/Set1_ground_truth_images_wo_CC/Canon1...,6.222635,107.081445,3.983099,7.666895
4,/content/Set1_input_images_wo_CC_JPG/NikonD40_...,/content/Set1_ground_truth_images_wo_CC/NikonD...,5.275175,49.799210,3.046508,5.924701
5,/content/Set1_input_images_wo_CC_JPG/FujifilmX...,/content/Set1_ground_truth_images_wo_CC/Fujifi...,4.609998,161.234188,5.724132,7.337883
6,/content/Set1_input_images_wo_CC_JPG/Canon600D...,/content/Set1_ground_truth_images_wo_CC/Canon6...,5.995913,163.388590,5.237102,9.366609
7,/content/Set1_input_images_wo_CC_JPG/IMG_0401_...,/content/Set1_ground_truth_images_wo_CC/IMG_04...,2.486857,17.407462,3.280565,2.684214
8,/content/Set1_input_images_wo_CC_JPG/IMG_0785_...,/content/Set1_ground_truth_images_wo_CC/IMG_07...,3.048299,18.659093,3.375651,3.273353
9,/content/Set1_input_images_wo_CC_JPG/Canon1DsM...,/content/Set1_ground_truth_images_wo_CC/Canon1...,2.699641,26.179680,2.027146,3.220935
